In [ ]:
# CÉLULA 1: IMPORTS GERAIS E DE ML
# Importar TODAS as ferramentas necessárias de uma só vez.
     

print("--- CÉLULA 1: Carregando todas as bibliotecas... ---")

# --- Bibliotecas de Coleta de Dados (ETL da API) ---

# 'requests' é a biblioteca que faz as requisições síncronas HTTP para a API do PNCP.
import requests

# 'sqlite3' biblioteca de banco de dados nativa e leve para operações SQL (os arquivos .db).
import sqlite3

# 'json' essencial realizar operações na resposta da API, transformar em Dicionário Python, facilitar leitura (dumps)
import json

# 'time' é usado para o `time.sleep()`, pausa no código pra não floodar a API (rate limiting).
import time

# 'datetime' é usado p/ obter data e hora atuais, útil para logs e timestamps no banco de dados.
from datetime import datetime


# --- Bibliotecas de Análise de Dados e ML ---

# 'pandas' ('pd') - Ferramenta principal de análise de dados
# Carrega, limpa e manipula os dados em tabelas (DataFrames).
import pandas as pd

# 'numpy' ('np') Biblioteca utilizada para cálculos matemáticos - pandas necessita dela no funcionamento
import numpy as np

# 're' é a biblioteca de "Regular Expressions" (Expressões Regulares)- limpeza e normalização de textos
import re

# 'joblib' é a ferramenta que usamos para gerar nosso modelo de ML treinado
# e salvá-lo em um arquivo (o 'modelo_relevancia.joblib').
import joblib 

# 'nltk' (Natural Language Toolkit) é a principal biblioteca de Processamento de Linguagem Natural.
# 'stopwords' é uma lista de palavras comuns ('de', 'a', 'o', 'com') que não têm valor para o modelo.
import nltk
from nltk.corpus import stopwords

# --- Bibliotecas do Scikit-learn (O "Cérebro" do ML) ---

# 'train_test_split' é a função que divide nosso "gabarito" em duas partes:
# 1. Dados de Treino (80%): O que o modelo vai "estudar".
# 2. Dados de Teste (20%): O que o modelo vai "fazer a prova" (dados desconhecidos pelo modelo).
from sklearn.model_selection import train_test_split

# 'TfidfVectorizer' serve basicamente para transformar uma lista de textos em uma 
# matriz de números de forma inteligente (TF-IDF) dando mais peso para palavras raras e 
# importantes (ex: "vazão" "medidor") e menos peso para palavras comuns (ex: "aquisição" "contratação").
from sklearn.feature_extraction.text import TfidfVectorizer

# 'SGDClassifier' (Stochastic Gradient Descent Classifier) é o algoritmo de ML escolhido.
# Ele é leve, rápido e muito eficaz para classificação de texto. É o "aluno" que vamos treinar.
from sklearn.linear_model import SGDClassifier

# 'Pipeline' é o "organizador". Ele nos permite combinar o uso do 'TfidfVectorizer' (tradutor)
# e o 'SGDClassifier' (aluno) em uma única execução. Isso garante que qualquer
# dado novo (na produção) passe exatamente pelo mesmo processo de tradução do treino.
from sklearn.pipeline import Pipeline

# 'classification_report' e 'accuracy_score' são as ferramentas de "nota".
# Elas geram o "boletim" (Relatório de Classificação) que nos diz o quão bem o modelo
# se saiu na prova (Precision, Recall, F1-score).
from sklearn.metrics import classification_report, accuracy_score

print("Bibliotecas importadas com sucesso.")

--- CÉLULA 1: Carregando todas as bibliotecas... ---
Bibliotecas importadas com sucesso.


In [3]:
# CÉLULA 2: CONFIGURAÇÕES GLOBAIS
# Define TODOS os nomes de arquivos, parâmetros e segredos em um lugar só.
# Mudar qualquer coisa aqui (ex: NOME_DB_PRODUCAO) muda o comportamento do script inteiro.

print("--- CÉLULA 2: Definindo todas as configurações... ---")

# --- 1. Configs da API e DB de Coleta ---

# Endereço base da API
BASE_URL = "https://pncp.gov.br/api/consulta"

# O caminho específico da API que queremos (Contratações com recebimento de proposta em aberto)
ENDPOINT = "/v1/contratacoes/proposta"

# Junção dos dois para formar o URL completo
URL_FINAL = BASE_URL + ENDPOINT

# Parâmetros de filtro para a coleta de PRODUÇÃO (399 páginas)
# Isso é o que dizemos à API: "Eu quero..."
params_coleta_producao = {
    'dataFinal': 20251216,                # "...licitações até esta data."
    'codigoModalidadeContratacao': 6,     # "...apenas Pregão." #6 Pregão / 8 #Dispensa
    'pagina': 1,                          # "...começando da página 1."
    'tamanhoPagina': 50                   # "...me dê 50 itens por página."
}

# Limite de paginação para ser usado na função
LIMITE_PAGINAS_PRODUCAO = 399


# Nome do DB onde os dados de PRODUÇÃO (os 19.9k) serão salvos
# A Célula 7 (Ação 1) vai escrever neste arquivo.
# A Célula 10 (Ação 4) vai ler deste arquivo.
NOME_DB_PRODUCAO = "pncp_data_19000.db"


# --- 2. Configs de TREINAMENTO ---

#  DB de 500 amostras pré-rotulado com a maioria dos '0's (irrelevantes)
NOME_DB_TREINO_NEGATIVOS = "pncp_data_500.db"

# Lista com IDX dos '1's (relevantes) que eu anotei manualmente do  DB de 500 de 0s Irrelevantes
# Posteriormente o é feito o rotulamento desses dados no Banco
IDS_RELEVANTES_CADERNO = [5, 85, 164, 199, 319, 327, 463]

# Datasets em arquivos CSV que contêm os '1's (relevantes) puros, rotulados manualmente
# dataset_2024 -> 94 relevantes
# dataset_2025 -> 105 relevantes
ARQUIVOS_CSV_POSITIVOS = ["dataset_2024.csv", "dataset_2025.csv"]


# O "Gabarito" final: O arquivo CSV que a Célula 8 (ETL) vai criar - Dataset combinado CSVs 2024 + 2025 + pncp_data_500 
NOME_ARQUIVO_GABARITO = "dataset.csv"

# O "Cérebro": O arquivo final que a Célula 9 (Treino) vai criar
NOME_ARQUIVO_MODELO = "modelo_relevancia.joblib"

# --- 3. Configs de PRODUÇÃO (Inferência/Filtro) ---

# O banco de dados final que a Célula 10 (Produção) vai criar,
# contendo apenas as licitações filtradas 
NOME_DB_FILTRADO = "licitacoes_filtradas_19k.db"

# Necessário processar por lote (batch) para não usar toda memória RAM
# Se não colocar esse limite trava o Kernel do Jupyter e falha a execução
#  Define Quantos itens do DB de 19.9k vamos processar de cada vez.
TAMANHO_DO_LOTE = 2000 


# --- 4. Configs do NLTK (Stopwords) ---

# Tenta carregar a lista de stopwords...
try:
    lista_stopwords = stopwords.words('portuguese')
# ...se falhar (porque nunca foi baixado)...
except LookupError:
     print("ERRO: Stopwords não carregadas. Baixando 'stopwords' do NLTK...")
     # ...baixa o pacote necessário...
     nltk.download('stopwords')
     # ...e tenta de novo.
     lista_stopwords = stopwords.words('portuguese')

# Converte a *lista* ['o', 'a', 'de'] em um *set* {'o', 'a', 'de'}.
# Isso faz a verificação (palavra in stop_words_pt) ser milhares de vezes mais rápida.
stop_words_pt = set(lista_stopwords)
print(f"Total de {len(stop_words_pt)} stopwords em português carregadas.")

print("Configurações concluídas.")

--- CÉLULA 2: Definindo todas as configurações... ---
Total de 207 stopwords em português carregadas.
Configurações concluídas.


In [4]:
# CÉLULA 3: DEFINIÇÃO DA FERRAMENTA - configurar_db
# Define o esquema do nosso banco de dados (table) no no SQLite.

# Essa Função só roda quando "chamar" configurar_db() no código, por isso é declarada antes
def configurar_db(nome_do_banco):
    """Cria a tabela 'contratacoes' no DB especificado se ela não existir."""
    
    # Bloco try/except para capturar erros de banco de dados
    try:
        # Conecta ao arquivo .db (ou cria se não existir)
        conn = sqlite3.connect(nome_do_banco)
        # Cria um "cursor", que é o objeto que executa os comandos SQL
        cursor = conn.cursor()
        
        print(f"Configurando banco de dados: {nome_do_banco}")
        
        # 1. Criar a tabela (só se ela não existir)
        # Usamos """ (aspas triplas) para um comando SQL de múltiplas linhas
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS contratacoes (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                pagina_coleta INTEGER,
                timestamp_coleta TEXT,
                
                -- Esta coluna vai guardar o JSON "cru", como um backup/data lake
                dados_json TEXT, 
                
                -- [JSON1] Colunas "Geradas"
                -- O próprio SQLite vai abrir o 'dados_json' e extrair esses campos.
                -- 'STORED' significa que o dado é salvo em disco, permitindo indexação.
                
                pncp_id TEXT AS (json_extract(dados_json, '$.numeroControlePNCP')) STORED,
                orgao_nome TEXT AS (json_extract(dados_json, '$.orgaoEntidade.razaosocial')) STORED,
                uf_sigla TEXT AS (json_extract(dados_json, '$.unidadeOrgao.ufSigla')) STORED,
                
                -- A coluna mais importante para nós: o texto do objeto
                objeto_compra TEXT AS (json_extract(dados_json, '$.objetoCompra')) STORED,
                
                -- Garante que o mesmo 'pncp_id' não seja inserido duas vezes
                UNIQUE(pncp_id) 
            );
        """)

        # 2. Criar IDXs (só se não existirem)
        # Um índice torna as buscas (SELECT) por esses campos muito mais rápidas.
        cursor.execute("CREATE INDEX IF NOT EXISTS idx_pncp_id ON contratacoes (pncp_id);")
        cursor.execute("CREATE INDEX IF NOT EXISTS idx_objeto_compra ON contratacoes (objeto_compra);")
        
        # Confirma (salva) as mudanças no banco de dados
        conn.commit()
        # Fecha a conexão com o arquivo
        conn.close()
        print(f"Banco '{nome_do_banco}' e tabela 'contratacoes' configurados.")
    
    # Se der erro (ex: disco cheio, permissão negada)
    except sqlite3.Error as e:
        print(f"ERRO ao configurar o DB {nome_do_banco}: {e}")

print("--- CÉLULA 3: Função 'configurar_db' definida. ---")

--- CÉLULA 3: Função 'configurar_db' definida. ---


In [5]:
# CÉLULA 4: DEFINIÇÃO DAS FERRAMENTAS - fetch_com_retry e inserir_dados
# Define a função que chama a API (fetch) e o função que insere os dados no banco.


def fetch_com_retry(url, params, max_tentativas=5, backoff_inicial=1):
    """Busca dados da API com retry e backoff exponencial."""
    
    # Cabeçalho HTTP simples
    headers = {'accept': '*/*'}
    tentativas = 0
    backoff = backoff_inicial
    
    # Loop de tentativas (vai tentar até 'max_tentativas')
    while tentativas < max_tentativas:
        try:
            # A chamada HTTP real. 'timeout=30' desiste se a API não responder em 30s.
            resposta = requests.get(url, params=params, headers=headers, timeout=30)
            
            # Código 200 = SUCESSO
            if resposta.status_code == 200:
                # Converte o texto (JSON) da resposta em um Dicionário Python
                dados_resposta = resposta.json()
                
                # Checagem de segurança: a resposta é um dict e tem a chave 'data'?
                if isinstance(dados_resposta, dict) and 'data' in dados_resposta:
                    
                    # Extrai a lista de licitações
                    dados_lista = dados_resposta.get('data', [])
                    
                    # Extrai o número total de páginas (para o log)
                    total_paginas = dados_resposta.get('totalPaginas') 
                    
                    # Retorna os dois valores (em uma tupla)
                    return (dados_lista, total_paginas)
                else:
                    # A API respondeu 200, mas o JSON veio zuado.
                    return ([], None)
            
            # Código 204 = Fim dos dados. A API diz "Ok, mas não tenho mais nada para te dar".
            if resposta.status_code == 204:
                return ([], None) # Sinaliza o fim.
            
            # Qualquer outro erro (ex: 500, 403), imprime e tenta de novo.
            print(f"Erro {resposta.status_code}. Tentando novamente em {backoff}s...")
        
        # Se a rede falhar (ex: Wi-Fi caiu)
        except requests.exceptions.RequestException as e:
            print(f"Erro de conexão: {e}. Tentando novamente em {backoff}s...")
        
        # Espera antes de tentar de novo
        time.sleep(backoff)
        tentativas += 1
        backoff *= 2 # "Backoff exponencial": espera 1s, 2s, 4s, 8s...
        
    print("Número máximo de tentativas atingido. Falha ao buscar dados.")
    return None # Retorna None em caso de falha total

def inserir_dados(nome_db, dados_pagina, pagina_coletada):
    """Insere uma lista de registros (uma página) no SQLite."""
    try:
        conn = sqlite3.connect(nome_db)
        cursor = conn.cursor()
        
        # Pega a hora atual para registrar quando o dado foi coletado
        timestamp_atual = datetime.now().strftime('%Y-%m-%d %H:%M:%S') 
        
        registros_inseridos = 0
        
        # Loop nos 50 itens da página
        for registro in dados_pagina:
            
            # Converte o dict Python de volta para uma string JSON para salvar no DB
            registro_json_str = json.dumps(registro)
            
            # Insere os dados. 'INSERT OR IGNORE' é o comando que pula duplicatas
            # (baseado na 'UNIQUE(pncp_id)' que definimos na Célula 3)
            cursor.execute("""
                INSERT OR IGNORE INTO contratacoes (pagina_coleta, timestamp_coleta, dados_json)
                VALUES (?, ?, ?);
            """, (pagina_coletada, timestamp_atual, registro_json_str))
            
            # 'cursor.rowcount' diz se a última linha foi inserida (1) ou ignorada (0)
            if cursor.rowcount > 0:
                registros_inseridos += 1
            
        conn.commit() # Salva as 50 inserções de uma vez
        print(f"   -> [DB] {registros_inseridos}/{len(dados_pagina)} registros novos salvos na página {pagina_coletada}.")
    except sqlite3.Error as e:
        print(f"   -> [ERRO DB] Falha ao inserir dados: {e}.")
    finally:
        if conn:
            conn.close()

print("--- CÉLULA 4: Funções 'fetch_com_retry' e 'inserir_dados' definidas. ---")

--- CÉLULA 4: Funções 'fetch_com_retry' e 'inserir_dados' definidas. ---


In [6]:
# CÉLULA 5: DEFINIÇÃO DA FERRAMENTA - buscar_dados_paginados
# Defineq a função que gerencia a coleta (fetch_com_retry) e inserção no db (inserir_dados).

# Esta função orquestra todo o processo de coleta
def buscar_dados_paginados(nome_do_banco, url_completa, parametros, limite_pagina=None):
    """Busca todos os dados da API e salva no DB especificado."""
    
    # 1. Garante que o DB e a tabela existam ANTES de começar.
    #    (Chama a função da Célula 3)
    configurar_db(nome_do_banco)

    # Inicia o contador de páginas
    pagina_atual = 1
    total_registros_processados = 0
    total_paginas_api = '???' # Começa sem saber o total
    
    print("--- INICIANDO BUSCA PAGINADA ---")
    if limite_pagina:
        print(f"--- Modo LIMITE DE PÁGINAS Ativado: Parar após página {limite_pagina} ---")
    
    # Loop infinito que só para com 'break'
    while True:
        
        # Checagem de segurança (limite de páginas)
        if limite_pagina is not None and pagina_atual > limite_pagina:
            print(f"\nLimite de páginas ({limite_pagina}) atingido. Parando a busca.")
            break

        # Prepara os filtros para a página ATUAL
        params_paginados = parametros.copy() # .copy() é vital para não zoar o original
        params_paginados['pagina'] = pagina_atual 
        print(f"\n>>>> BUSCANDO PÁGINA: {pagina_atual} de {total_paginas_api}")
        
        # 2. Chama o "robô" (definido na Célula 4)
        retorno_fetch = fetch_com_retry(url_completa, params_paginados)
        
        # Se o robô falhar (retornar None), aborta tudo
        if retorno_fetch is None:
            print("Erro (fetch_com_retry retornou None). FIM DA BUSCA.")
            break 
            
        # "Desempacota" a tupla (lista_de_dados, total_de_paginas)
        dados_pagina, total_paginas_resposta = retorno_fetch
        
        # Atualiza o total de páginas (só na primeira vez, ou se mudar)
        if total_paginas_resposta is not None:
            total_paginas_api = total_paginas_resposta

        # Condição de parada: Se a API retornar uma lista vazia, acabaram os dados.
        if not dados_pagina: 
            print(f"Fim dos dados (lista vazia) na página {pagina_atual}. FIM DA BUSCA.")
            break 
            
        # 4. PERSISTÊNCIA: Chama o "pedreiro" (definido na Célula 4)
        inserir_dados(nome_do_banco, dados_pagina, pagina_atual)
        total_registros_processados += len(dados_pagina)

        # 5. Incrementa o contador para a próxima página
        pagina_atual += 1
        # Pausa de meio segundo para ser "educado" com a API
        time.sleep(0.5) 
        
    print("\n--- BUSCA CONCLUÍDA ---")
    print(f"Total de registros processados: {total_registros_processados}")
    return True

print("--- CÉLULA 5: Função 'buscar_dados_paginados' definida. ---")

--- CÉLULA 5: Função 'buscar_dados_paginados' definida. ---


In [7]:
# CÉLULA 6: DEFINIÇÃO DA FERRAMENTA - preprocessar_texto
# Digere os dados brutos através de Expressões Regulares e Tokenização

def preprocessar_texto(texto):
    """Limpa e normaliza uma string de texto para o ML."""
    
    # Etapa 1: Checagem de Segurança
    # Se o 'objeto_compra' for Nulo ou um BLOB, ele não será 'str'.
    # Retornamos uma string vazia para evitar erros.
    if not isinstance(texto, str):
        return ""

    # Etapa 2: Normalização
    # Converte "AQUISIÇÃO" para "aquisição"
    texto_etapa2 = texto.lower()
    
    # Etapa 3: Limpeza de Ruído (Regex)
    # [^a-zÀ-ú\\s] : "encontre qualquer caractere que NÃO seja (^)
    # uma letra de a-z, OU uma letra acentuada (À-ú), OU (\\) um espaço (s)"
    # e substitua por um espaço em branco ' '.
    texto_etapa3 = re.sub(r'[^a-zÀ-ú\s]', ' ', texto_etapa2, flags=re.IGNORECASE)
    
    # '\\s+' : "encontre um ou mais (s+) espaços em sequência"
    # e substitua por um espaço único ' '.
    texto_etapa3 = re.sub(r'\\s+', ' ', texto_etapa3).strip() # .strip() remove espaços no início/fim
    
    # Etapa 4: Tokenização (Virar uma lista de palavras)
    # Encontra todas as "palavras" (sequências de letras)
    tokens_etapa4 = re.findall(r'\b[a-zÀ-ú]+\b', texto_etapa3)
    
    # Etapa 5: Remoção de Stopwords
    # List comprehension:
    # "Para cada 'palavra' na minha lista 'tokens_etapa4'..."
    tokens_etapa5 = [palavra for palavra in tokens_etapa4 
                     # "...mantenha-a SOMENTE SE..."
                     # "...ela NÃO estiver na minha lista de stopwords (definida na Célula 2)..."
                     if palavra not in stop_words_pt 
                     # "...E (and) o tamanho dela for maior que 2." (remove 'i', 'é', 'se')
                     and len(palavra) > 2]
    
    # Etapa 6: Remontagem da String
    # Junta a lista de palavras limpas de volta em uma string, separada por espaço.
    # Ex: ['medidor', 'vazao'] -> "medidor vazao"
    return " ".join(tokens_etapa5)

print("--- CÉLULA 6: Função 'preprocessar_texto' definida. ---")

--- CÉLULA 6: Função 'preprocessar_texto' definida. ---


In [ ]:
# CÉLULA 7: [AÇÃO 1] - DEMONSTRAÇÃO DA COLETA DE DADOS
# WTF: Rodando uma coleta PEQUENA (3 páginas) para provar que o "Pescador" funciona.
#      Vamos salvar em um DB de "demo" separado para não zoar os arquivos de produção.

print("--- CÉLULA 7: INICIANDO DEMONSTRAÇÃO DA COLETA ---")

# --- 1. Definir configurações LOCAIS para este teste ---

# Nome do arquivo de banco de dados SÓ PARA ESTE TESTE
NOME_DB_DEMO = "pncp_data_DEMO_3pag.db" 

# Limite de páginas para o teste (como solicitado: 3)
LIMITE_PAG_DEMO = 3

# Parâmetros customizados para o teste
# Vamos pegar licitações de DISPENSA (cód 8) só para ser diferente
params_demo = {
    'dataFinal': 20251216,                
    'codigoModalidadeContratacao': 6,
    'pagina': 1,                          
    'tamanhoPagina': 50                   
}

print(f"\n[DEMO] Salvando dados em: '{NOME_DB_DEMO}'")
print(f"[DEMO] Parâmetros: {params_demo}")
print(f"[DEMO] Limite de páginas: {LIMITE_PAG_DEMO}")
print("\nChamando a função 'buscar_dados_paginados' (definida na Célula 5)...")

# --- 2. Chamar o "Chefe" (definido na Célula 5) com as configs de DEMO ---
# Usamos as variáveis locais que acabamos de criar.
try:
    buscar_dados_paginados(
        NOME_DB_DEMO, 
        URL_FINAL, 
        params_demo, 
        LIMITE_PAG_DEMO
    )
    print("\n--- DEMONSTRAÇÃO DA COLETA CONCLUÍDA ---")
    print(f"Arquivo '{NOME_DB_DEMO}' foi criado no seu diretório.")
    print(f"Ele contém aprox. {LIMITE_PAG_DEMO * 50} (ou menos) licitações do tipo 'Pregão'.")

except Exception as e:
    print(f"\nERRO durante a coleta de demonstração: {e}")

--- CÉLULA 7: INICIANDO DEMONSTRAÇÃO DA COLETA ---

[DEMO] Salvando dados em: 'pncp_data_DEMO_3pag.db'
[DEMO] Parâmetros: {'dataFinal': 20251216, 'codigoModalidadeContratacao': 6, 'pagina': 1, 'tamanhoPagina': 50}
[DEMO] Limite de páginas: 3

Chamando a função 'buscar_dados_paginados' (definida na Célula 5)...
Configurando banco de dados: pncp_data_DEMO_3pag.db
Banco 'pncp_data_DEMO_3pag.db' e tabela 'contratacoes' configurados.
--- INICIANDO BUSCA PAGINADA ---
--- Modo LIMITE DE PÁGINAS Ativado: Parar após página 3 ---

>>>> BUSCANDO PÁGINA: 1 de ???
   -> [DB] 50/50 registros novos salvos na página 1.

>>>> BUSCANDO PÁGINA: 2 de 400
   -> [DB] 50/50 registros novos salvos na página 2.

>>>> BUSCANDO PÁGINA: 3 de 400
   -> [DB] 50/50 registros novos salvos na página 3.

Limite de páginas (3) atingido. Parando a busca.

--- BUSCA CONCLUÍDA ---
Total de registros processados: 150

--- DEMONSTRAÇÃO DA COLETA CONCLUÍDA ---
Arquivo 'pncp_data_DEMO_3pag.db' foi criado no seu diretório.
Ele 

In [7]:
# CÉLULA 8: [AÇÃO 2] - RODAR O ETL DE TREINAMENTO
# APERTE AQUI para criar o "gabarito" ('dataset.csv').
# Junta os CSVs 2024 e 205 (1s) e o DB de 500 (Maioria 0s/1s) num arquivo só.

print("--- CÉLULA 8: INICIANDO ETL DE TREINAMENTO (Criação do Gabarito) ---")

try:
    # --- 1. Carregar dados POSITIVOS (1s) dos CSVs ---
    # Pega o nome dos arquivos (ex: ["dataset_2024.csv", ...]) da Célula 2
    print(f"Carregando 1s de: {ARQUIVOS_CSV_POSITIVOS}")
    
    # Cria uma lista vazia para guardar os DataFrames (tabelas)
    lista_dfs_positivos = []
    
    # Loop nos nomes dos arquivos
    for arquivo in ARQUIVOS_CSV_POSITIVOS:
        
        # Lê o CSV, mas 'usecols' pega SÓ as colunas 'x' e 'y' (ignora 'texto_bruto')
        df_temp = pd.read_csv(arquivo, usecols=['x', 'y'], dtype={'y': str})
        
        # Adiciona a tabela lida à nossa lista
        lista_dfs_positivos.append(df_temp)

    # 'concat' empilha os DataFrames da lista (2024 + 2025) em um só
    df_positivos = pd.concat(lista_dfs_positivos, ignore_index=True)
    
    # Renomeia as colunas 'x' -> 'Objeto' e 'y' -> 'Relevante'
    df_positivos = df_positivos.rename(columns={'x': 'Objeto', 'y': 'Relevante'})
    
    # Garante que todos sejam '1', por segurança
    df_positivos['Relevante'] = 1 
    print(f"Total de {len(df_positivos)} exemplos POSITIVOS (1) carregados dos CSVs.")

    # --- 2. Carregar e Rotular dados do DB de 500 (Negativos) ---
    # Pega o nome do DB de 500 amostras da Célula 2
    print(f"Carregando e rotulando dados do '{NOME_DB_TREINO_NEGATIVOS}'...")
    conn = sqlite3.connect(NOME_DB_TREINO_NEGATIVOS)
    
    # Query para pegar o ID (para rotular) e o objeto_compra (o texto)
    query_500 = "SELECT id, objeto_compra FROM contratacoes"
    df_500 = pd.read_sql_query(query_500, conn) # Lê o DB para um DataFrame
    conn.close()
    print(f"Carregados {len(df_500)} itens do DB de treino.")

    # [SOLUÇÃO BLOBs 1] Remove linhas onde 'objeto_compra' é Nulo (None)
    df_500 = df_500.dropna(subset=['objeto_compra'])
    
    # [SOLUÇÃO BLOBs 2] Remove linhas onde o texto é literalmente a palavra "BLOB"
    # O '~' significa NÃO (inverte a seleção)
    df_500 = df_500[~df_500['objeto_compra'].str.contains('BLOB', na=False, case=False)]
    print(f"Removidos BLOBs/Nulos. Restam {len(df_500)} itens válidos.")

    # [SOLUÇÃO Labels Manuais] Define a função-dicionário (o "caderno")
    def rotular_do_caderno(id):
        # IDS_RELEVANTES_CADERNO foi definido na Célula 2
        return 1 if id in IDS_RELEVANTES_CADERNO else 0
            
    # '.apply()' roda a função 'rotular_do_caderno' para cada 'id' no DataFrame
    df_500['Relevante'] = df_500['id'].apply(rotular_do_caderno)
    
    # Renomeia a coluna de texto para bater com o df_positivos
    df_500 = df_500.rename(columns={'objeto_compra': 'Objeto'})
    print("Rotulagem manual do DB de 500 concluída.")
    
    # --- 3. Combinar TUDO ---
    print("Combinando todos os datasets...")
    
    # Empilha o 'df_positivos' e o 'df_500' (só as colunas 'Objeto' e 'Relevante')
    df_completo = pd.concat([df_positivos, df_500[['Objeto', 'Relevante']]], ignore_index=True)
    
    # Remove duplicatas de 'Objeto'.
    # 'keep='first'' é crucial: se um objeto estava nos CSVs (1s) E no DB (0s),
    # ele mantém o PRIMEIRO que viu (o '1' do CSV), garantindo a qualidade dos dados.
    df_final = df_completo.drop_duplicates(subset=['Objeto'], keep='first')
    
    # --- 4. Salvar o "Gabarito" Final ---
    # Salva o DataFrame final no 'dataset.csv' (nome definido na Célula 2)
    df_final.to_csv(NOME_ARQUIVO_GABARITO, index=False, encoding='utf-8-sig')
    
    print(f"\n--- SUCESSO! (ETL de Treino) ---")
    print(f"Arquivo final '{NOME_ARQUIVO_GABARITO}' criado com {len(df_final)} exemplos.")
    print("Balanceamento final do dataset (limpo e revisado):")
    
    # Mostra quantos 0s e 1s temos no total
    print(df_final['Relevante'].value_counts())

except Exception as e:
    print(f"ERRO CRÍTICO no ETL de Treino (Célula 8): {e}")

--- CÉLULA 8: INICIANDO ETL DE TREINAMENTO (Criação do Gabarito) ---
Carregando 1s de: ['dataset_2024.csv', 'dataset_2025.csv']
Total de 197 exemplos POSITIVOS (1) carregados dos CSVs.
Carregando e rotulando dados do 'pncp_data_500.db'...
Carregados 500 itens do DB de treino.
Removidos BLOBs/Nulos. Restam 500 itens válidos.
Rotulagem manual do DB de 500 concluída.
Combinando todos os datasets...

--- SUCESSO! (ETL de Treino) ---
Arquivo final 'dataset.csv' criado com 672 exemplos.
Balanceamento final do dataset (limpo e revisado):
Relevante
0    475
1    197
Name: count, dtype: int64


In [8]:
# CÉLULA 9: [AÇÃO 3] - RODAR O TREINAMENTO
# WTF: APERTE AQUI para "estudar" o 'dataset.csv' e salvar o "cérebro".
#      (Célula 11 original, com o 'class_weight' de elite)

print("--- CÉLULA 9: INICIANDO TREINAMENTO DO MODELO ---")
# Pega os nomes de arquivos da Célula 2
print(f"Lendo gabarito: '{NOME_ARQUIVO_GABARITO}'")
print(f"Salvando cérebro em: '{NOME_ARQUIVO_MODELO}'")

try:
    # --- 1. Carregar o Gabarito ---
    # Lê o arquivo 'dataset.csv' que a Célula 8 acabou de criar
    df_treino = pd.read_csv(NOME_ARQUIVO_GABARITO)
    print(f"\nCarregados {len(df_treino)} exemplos (licitações) do gabarito.")
    print("Balanceamento real:")
    print(df_treino['Relevante'].value_counts())
    
    # --- 2. Pré-processamento (Limpeza) ---
    print("\nAplicando 'preprocessar_texto'...")
    # Chama a função "liquidificador" (definida na Célula 6) para cada linha da coluna 'Objeto'
    df_treino['objeto_limpo'] = df_treino['Objeto'].apply(preprocessar_texto)
    
    # --- 3. Separar X (Features) e y (Labels) ---
    # X = O que o modelo lê (o texto limpo)
    X = df_treino['objeto_limpo']
    # y = A resposta correta (o gabarito 0 ou 1)
    y = df_treino['Relevante']
    
    # --- 4. Dividir em Treino/Teste ---
    print("Dividindo em Treino (80%) e Teste (20%)...")
    # 'test_size=0.2' = 20% para "prova" (teste)
    # 'random_state=42' = garante que a divisão seja sempre a mesma (reprodutibilidade)
    # 'stratify=y' = Garante que os 20% de teste tenham a *mesma proporção* de 0s e 1s
    #                  que o dataset original. Muito importante!
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    
    # --- 5. Definir o Pipeline (A "Linha de Montagem" de Elite) ---
    modelo_pipeline = Pipeline([
        # Passo 1: O "Tradutor" (TF-IDF)
        ('tfidf', TfidfVectorizer(max_features=1000,    # Considera só as 1000 palavras mais importantes
                                  ngram_range=(1, 2))), # Olha "vazão" (1) e "medidor vazão" (2)
        
        # Passo 2: O "Aluno" (Classificador)
        ('clf', SGDClassifier(loss='hinge',              # 'hinge' é o "motor" (SVM), ótimo para isso
                             random_state=42,          # Para reprodutibilidade
                             max_iter=100,             # Número de "passadas" de estudo
                             class_weight='balanced')) # A MÁGICA: "Não seja preguiçoso!"
    ])
    print("Pipeline definido com 'class_weight=balanced'.")

    # --- 6. TREINAR ---
    print("\n--- TREINANDO O MODELO (método .fit()) ---")
    # Manda a "Linha de Montagem" (Pipeline) "estudar" os dados de treino
    modelo_pipeline.fit(X_train, y_train)
    print("TREINAMENTO CONCLUÍDO.")

    # --- 7. AVALIAR ---
    print("\n--- AVALIANDO O MODELO (nos 20% de teste) ---")
    # Manda o modelo treinado "fazer a prova" (prever os 20% que ele nunca viu)
    y_pred = modelo_pipeline.predict(X_test)
    
    print("\nRelatório de Classificação Detalhado:")
    # Imprime o "Boletim": compara as previsões (y_pred) com o gabarito (y_test)
    print(classification_report(y_test, y_pred, target_names=['Irrelevante (0)', 'Relevante (1)']))

    # --- 8. SALVAR O CÉREBRO ---
    print(f"\n--- Salvando o modelo treinado em '{NOME_ARQUIVO_MODELO}' ---")
    # "Congela" o modelo treinado e salva em um arquivo
    joblib.dump(modelo_pipeline, NOME_ARQUIVO_MODELO)
    print("Modelo salvo com sucesso!")

# Se o 'dataset.csv' não existir...
except FileNotFoundError:
    print(f"ERRO CRÍTICO: Arquivo '{NOME_ARQUIVO_GABARITO}' não encontrado.")
    print("Rode a CÉLULA 8 (ETL de Treino) primeiro.")
except Exception as e:
    print(f"Ocorreu um erro inesperado no Treino: {e}")

--- CÉLULA 9: INICIANDO TREINAMENTO DO MODELO ---
Lendo gabarito: 'dataset.csv'
Salvando cérebro em: 'modelo_relevancia.joblib'

Carregados 672 exemplos (licitações) do gabarito.
Balanceamento real:
Relevante
0    475
1    197
Name: count, dtype: int64

Aplicando 'preprocessar_texto'...
Dividindo em Treino (80%) e Teste (20%)...
Pipeline definido com 'class_weight=balanced'.

--- TREINANDO O MODELO (método .fit()) ---
TREINAMENTO CONCLUÍDO.

--- AVALIANDO O MODELO (nos 20% de teste) ---

Relatório de Classificação Detalhado:
                 precision    recall  f1-score   support

Irrelevante (0)       0.98      0.98      0.98        95
  Relevante (1)       0.95      0.95      0.95        40

       accuracy                           0.97       135
      macro avg       0.96      0.96      0.96       135
   weighted avg       0.97      0.97      0.97       135


--- Salvando o modelo treinado em 'modelo_relevancia.joblib' ---
Modelo salvo com sucesso!


In [ ]:
# CÉLULA 10: [AÇÃO 4] - RODAR A PRODUÇÃO (O FILTRO)
# WTF: APERTE AQUI para usar o "cérebro" e filtrar seu DB de 19.9k.
#      (Célula 12 original, com o processamento em lotes)

print("--- CÉLULA 10: INICIANDO APLICAÇÃO EM PRODUÇÃO (FILTRO) ---")
# Pega os nomes de arquivos da Célula 2
print(f"Carregando modelo treinado: '{NOME_ARQUIVO_MODELO}'")
print(f"Lendo DB de Produção: '{NOME_DB_PRODUCAO}'")
print(f"Salvando licitações filtradas em: '{NOME_DB_FILTRADO}'")

try:
    # --- 1. Carregar o Cérebro ---
    print(f"\nCarregando modelo '{NOME_ARQUIVO_MODELO}'...")
    # Carrega o 'modelo_relevancia.joblib' que a Célula 9 criou
    modelo_carregado = joblib.load(NOME_ARQUIVO_MODELO)
    print("Modelo carregado com sucesso.")

    # --- 2. Ler DB de Produção (em lotes) ---
    print(f"Conectando ao SQLite '{NOME_DB_PRODUCAO}' (em lotes de {TAMANHO_DO_LOTE})...")
    conn = sqlite3.connect(NOME_DB_PRODUCAO)
    
    # Pega todas as colunas que queremos no resultado final
    query_sql = "SELECT id, pncp_id, orgao_nome, uf_sigla, objeto_compra, dados_json FROM contratacoes"
    
    # Lista vazia para guardar as licitações filtradas de cada lote
    lista_de_resultados = []
    total_processado = 0
    total_relevante = 0

    # [Prevenir o Limite de uso Da Memória RAM]
    # 'pd.read_sql_query' com 'chunksize' não lê o DB todo.
    # Ele cria um "iterador" que lê o DB em pedaços (lotes - chunks) de 2000 linhas.
    for chunk_df in pd.read_sql_query(query_sql, conn, chunksize=TAMANHO_DO_LOTE):
        
        total_processado += len(chunk_df)
        print(f"\n--- Processando lote... (Itens até {total_processado}) ---")
        
        # 3a. Limpeza do Lote (filtro de BLOBs)
        chunk_limpo = chunk_df.dropna(subset=['objeto_compra']).copy()
        chunk_limpo = chunk_limpo[~chunk_limpo['objeto_compra'].str.contains('BLOB', na=False, case=False)]
        
        if len(chunk_limpo) == 0:
            print("Lote vazio após limpeza. Pulando...")
            continue
            
        # 3b. Predição do Lote
        # Aplica o preprocessamento de teto (Célula 6) no lote
        chunk_limpo['objeto_limpo'] = chunk_limpo['objeto_compra'].apply(preprocessar_texto)
        
        # Usa o modelo treinado (carregado) para classificar o lote
        predicoes = modelo_carregado.predict(chunk_limpo['objeto_limpo'])
        chunk_limpo['relevante_pred'] = predicoes
        
        # 3c. Filtragem do Lote
        # Pega só as linhas que o modelo disse que são '1'
        df_relevante_do_lote = chunk_limpo[chunk_limpo['relevante_pred'] == 1]
        
        if len(df_relevante_do_lote) > 0:
            print(f"==> {len(df_relevante_do_lote)} itens RELEVANTES encontrados neste lote.")
            # Adiciona as licitações filtradas do lote na nossa lista de resultados
            lista_de_resultados.append(df_relevante_do_lote)
            total_relevante += len(df_relevante_do_lote)
        else:
            print("Nenhum item relevante neste lote.")
            
    conn.close()
    print("\n--- Todos os lotes foram processados. ---")

    # --- 4. SALVAMENTO DO RESULTADO FINAL ---
    if not lista_de_resultados:
        print("ALERTA: Nenhum item relevante foi encontrado no banco de dados inteiro.")
    
    else:
        # 'concat' junta as licitações filtradas de todos os lotes em um DataFrame final
        df_relevante_final = pd.concat(lista_de_resultados, ignore_index=True)
        
        print(f"\n--- RESULTADO DA FILTRAGEM TOTAL ---")
        print(f"De {total_processado} itens totais, {total_relevante} foram classificados como RELEVANTES.")

        print(f"\n--- Salvando os dados filtrados em '{NOME_DB_FILTRADO}' ---")
        # Define as colunas que queremos no DB final (excluindo 'objeto_limpo', etc.)
        colunas_finais = ['id', 'pncp_id', 'orgao_nome', 'uf_sigla', 'objeto_compra', 'dados_json']
        df_para_salvar = df_relevante_final[colunas_finais]
        
        # Cria um novo DB de saída
        conn_out = sqlite3.connect(NOME_DB_FILTRADO)
        # Salva o DataFrame final na tabela 'licitacoes_filtradas'
        df_para_salvar.to_sql('licitacoes_filtradas', conn_out, if_exists='replace', index=False)
        conn_out.close()
        
        print(f"\nArquivo '{NOME_DB_FILTRADO}' salvo com sucesso!")
        print("Amostra das licitações filtradas (primeiras 10 linhas):")
        print(df_para_salvar[['pncp_id', 'objeto_compra']].head(10))

except FileNotFoundError:
    print(f"ERRO CRÍTICO: Modelo '{NOME_ARQUIVO_MODELO}' ou DB '{NOME_DB_PRODUCAO}' não encontrado.")
    print("Rode a CÉLULA 9 (Treinamento) primeiro.")
except Exception as e:
    print(f"Ocorreu um erro inesperado: {e}")
    print(f"Verifique se a CÉLULA 6 ('preprocessar_texto') foi rodada.")

print("\n--- CÉLULA 10 (Produção) concluída ---")

--- CÉLULA 10: INICIANDO APLICAÇÃO EM PRODUÇÃO (FILTRO) ---
Carregando cérebro: 'modelo_relevancia.joblib'
Lendo DB de Produção: 'pncp_data_19000.db'
Salvando filé em: 'licitacoes_filtradas_19k.db'

Carregando modelo 'modelo_relevancia.joblib'...
Modelo carregado com sucesso.
Conectando ao SQLite 'pncp_data_19000.db' (em lotes de 2000)...

--- Processando lote... (Itens até 2000) ---
==> 78 itens RELEVANTES encontrados neste lote.

--- Processando lote... (Itens até 4000) ---
==> 92 itens RELEVANTES encontrados neste lote.

--- Processando lote... (Itens até 6000) ---
==> 108 itens RELEVANTES encontrados neste lote.

--- Processando lote... (Itens até 8000) ---
==> 97 itens RELEVANTES encontrados neste lote.

--- Processando lote... (Itens até 10000) ---
==> 101 itens RELEVANTES encontrados neste lote.

--- Processando lote... (Itens até 12000) ---
==> 110 itens RELEVANTES encontrados neste lote.

--- Processando lote... (Itens até 14000) ---
==> 87 itens RELEVANTES encontrados neste l